In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import casadi as ca

In [53]:
def h1(x):
    return ca.vertcat(*[ca.norm_2(x[1]**2+x[2]**2-1),
                        ca.norm_2(x[0]-0)])
    

#def h2(x):
#    return ca.norm_2(x-np.array([0,1,0]))



In [67]:
#h1(np.array([1.1, 3.1, 1.1]))
x_sym = ca.SX.sym('x_sym',3)
h1_sym = h1(x_sym)
h2_sym = h2(x_sym)
h1_jac = ca.jacobian(h1_sym, x_sym)
h2_jac = ca.jacobian(h2_sym, x_sym)
h1_jac_fn = ca.Function('h1_jac', [x_sym],[h1_jac])
h2_jac_fn = ca.Function('h2_jac', [x_sym],[h2_jac])
print(h1_sym)
print(h2_sym)
print(h1_jac_fn(np.array([1, 1, 0.03])))

h1_jac_pinv = ca.pinv(h1_jac)
h1_jac_pinv_fn = ca.Function('h1_jac_pinv_fn', [x_sym], [h1_jac_pinv])

h2_jac_pinv = ca.pinv(h2_jac)
h2_jac_pinv_fn = ca.Function('h2_jac_pinv_fn', [x_sym], [h2_jac_pinv])


[fabs(((sq(x_sym_1)+sq(x_sym_2))-1)), fabs(x_sym_0)]
sqrt(((sq(x_sym_0)+sq((x_sym_1-1)))+sq(x_sym_2)))

[[00, 2, 0.06], 
 [1, 00, 00]]


In [85]:
#A*x=b
#J*lam=F

F = np.array([2, 1, 1])
x = np.array([2,0,1.3])
pi1_val = h1_jac_pinv_fn(x).full()
pi2_val = h2_jac_pinv_fn(x).full()

#print(h1_jac_fn(x))
#print(pi_val.size)
#print(F.size)
#print(h1_jac_fn(x).full().shape)

print(ca.norm_2(F-h1_jac_fn(x).T@(F@pi1_val)).full())
#print(ca.norm_2(F-h2_jac_fn(x)@(F@pi2_val)).full())


[[1.]]


In [33]:
### Rotation test
from rotation_helpers import *

#r = np.array([0, np.pi, 0])
r = np.array([-0.3, 3*np.pi/5, -.8])
#r = ca.SX.sym('r', 3)

rot = xyz_to_rotation(r)
print(rot)
print(rot@(rot.T))
r2 = rotation_to_xyz(rot)
#print(r)
print(r-r2)
rot2 = xyz_to_rotation(r2)
print(rot-rot2)

x = np.array([0, 2, 3, .3, 1, 0])
T = xyz_pose_to_tmat(x)
print(T)
x2 = tmat_to_xyz_pose(T)
print(x-x2)

#R = rotvec_to_rotation(r)
#print(R)
#r2 = rotation_to_rotvec(R)
#print(r2)
#R2 = rotvec_to_rotation(r2)
#print(R-R2)


[[-0.215294, -0.221675, 0.951057], 
 [-0.88113, 0.463972, -0.0913208], 
 [-0.42102, -0.857666, -0.295215]]
@1=1, @2=0, @3=1.07553e-16, 
[[@1, @2, @2], 
 [@2, @1, @3], 
 [@2, @3, 1]]
@1=-3.14159, [@1, 0.628319, @1]
@1=0, 
[[@1, -8.32667e-17, @1], 
 [-1.11022e-16, -1.66533e-16, -2.77556e-17], 
 [1.11022e-16, @1, -5.55112e-17]]
@1=0, 
[[0.540302, @1, 0.841471, @1], 
 [0.248672, 0.955336, -0.15967, 2], 
 [-0.803888, 0.29552, 0.516171, 3], 
 [00, 00, 00, 1]]
@1=0, [@1, @1, @1, @1, @1, @1]
